<a href="https://colab.research.google.com/github/reyllama/TTIO/blob/master/04_Rescaling_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.autograd import Variable
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = "/content/drive/My Drive/corr1.csv"
df = pd.read_csv(path)
df = df.set_index(keys='stock_id')
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,000020,000040,000050,000060,000070,000075,000080,000087,000100,000105,000120,000140,000145,000150,000155,000157,000180,000210,000215,000220,000225,000227,000230,000240,000250,000270,000300,000320,000325,000370,000390,000400,000430,000440,000480,000490,000500,000520,000540,000545,...,95500,95570,95610,95660,95700,95720,95910,96040,96240,96300,96350,96530,96610,96630,96640,96690,96760,96770,96775,96870,97230,97520,97780,97800,97870,97950,97955,98120,98460,98660,99190,99220,99320,99340,99350,99410,99440,99520,99660,99830
stock_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000020,1.000000,0.221514,-0.037755,-0.021159,0.230919,0.259171,0.176753,0.195048,0.215654,0.175516,0.208703,0.139837,0.071031,0.116629,0.085471,0.165149,0.124488,0.078928,0.049496,0.537236,0.512802,0.303855,0.483129,0.017378,0.482491,-0.072407,0.017935,0.133918,0.041311,0.148041,0.110374,0.294617,0.040400,0.118770,-0.026300,0.162403,0.135262,0.439909,0.150139,0.018031,...,0.072378,0.177056,0.094984,0.257213,0.118036,0.061175,0.049229,-0.120927,0.027107,0.229460,0.137070,0.333912,0.253663,-0.004647,0.073918,0.051170,0.436779,0.070719,0.038662,-0.009037,-0.067703,0.160585,0.272692,0.061822,0.225241,0.039099,-0.051827,0.112372,0.146558,0.079731,0.119611,0.284219,0.065122,-0.193870,-0.055491,0.170469,0.298094,0.101488,0.083284,0.116734
000040,0.221514,1.000000,-0.084175,0.125700,0.048086,0.144288,-0.026518,0.139692,0.059997,0.230597,0.052476,0.017729,0.078381,0.209605,0.249582,0.245019,0.194707,0.179482,0.216908,0.249301,0.259880,0.179810,0.216462,0.096105,0.369667,0.131545,0.340488,0.217145,-0.060981,0.220581,0.082751,0.271047,0.300337,0.120322,0.053789,0.174838,0.239307,0.215961,0.197419,0.051451,...,0.414345,0.101782,0.059960,0.227716,0.063983,0.198506,0.071940,-0.059623,0.270323,0.338363,0.194589,0.144394,0.157667,0.114744,0.003995,0.002977,0.296121,0.238160,0.193186,0.199345,0.176108,0.322742,0.215993,0.116542,0.265938,-0.006009,-0.078675,0.207658,0.136085,0.270371,0.137788,0.219789,0.093931,-0.145495,-0.084699,0.106058,0.374286,0.271006,0.171826,0.207319
000050,-0.037755,-0.084175,1.000000,-0.026910,0.194396,0.145192,0.064836,0.065237,0.176480,0.184645,0.139954,0.074693,-0.034289,0.035887,0.030783,-0.029888,0.065450,-0.033895,0.003912,-0.018252,-0.050283,-0.105596,0.237094,0.159021,0.055958,-0.161801,0.105222,0.147823,0.026685,0.059532,0.117234,0.064063,0.016387,-0.015661,0.142240,0.177168,-0.066760,0.106400,0.167397,0.047480,...,0.066808,-0.015634,-0.007347,0.165473,0.041894,0.193574,0.004915,0.085066,0.082473,-0.012954,-0.084595,0.112413,0.189654,0.146540,-0.075025,0.157147,-0.006074,0.014898,0.053050,0.100744,0.187599,0.086372,-0.100047,-0.065204,-0.016283,0.050333,0.018721,0.032135,0.194629,-0.034360,0.241911,-0.094872,0.076623,0.101066,-0.186058,0.078683,0.081244,-0.111201,-0.024860,-0.019401
000060,-0.021159,0.125700,-0.026910,1.000000,0.178085,0.176415,0.025524,0.103304,0.094681,0.087511,0.031110,0.115337,0.369805,0.158645,0.135941,0.104001,0.026601,0.296269,0.305429,0.199097,0.127579,0.156409,-0.046473,0.031214,0.126763,0.006368,0.272023,0.066901,0.042691,0.142887,0.253146,0.276613,0.158297,0.213465,0.280961,0.082161,0.054212,0.177353,0.171967,0.140361,...,0.249084,0.048593,0.146570,0.088966,0.125445,0.113970,0.061257,-0.018510,0.115044,0.178463,0.144524,0.098360,-0.042493,0.190262,0.153143,0.074302,0.129936,0.167621,0.240689,0.047773,0.158049,0.017242,0.192288,0.098559,0.234262,0.172853,0.153594,0.259511,0.211723,-0.002448,0.289181,0.101877,0.057986,0.118551,0.005246,0.018182,0.133223,0.102192,0.063812,0.060242
000070,0.230919,0.048086,0.194396,0.178085,1.000000,0.658519,0.103320,0.213751,0.344049,0.187832,0.215465,0.226021,0.029073,0.179910,0.198224,0.239788,0.111998,0.209418,0.262103,0.363887,0.244465,0.167201,0.351404,0.071581,0.286885,-0.196303,0.235813,0.029320,0.166511,0.166894,0.243878,0.293949,0.183960,0.070419,0.150535,0.236201,0.186679,0.268939,0.107885,0.221739,...,0.349720,0.142874,0.008797,0.209065,0

In [3]:
dtype = torch.FloatTensor
epochs = 50
emb_dim = 32
embeddings = Variable(torch.randn(emb_dim, len(df)).type(dtype), requires_grad=True)
lr = 0.005
optimizer = optim.SGD([embeddings], lr=lr)
mean = np.abs(df).mean().mean()
coef = 1/mean
loss_track = []

for epoch in range(epochs):
  loss = 0
  for i in range(len(df)):
    v1 = embeddings[:, i].view(emb_dim, -1)
    dist = torch.norm(v1 - embeddings, dim=0).view(len(df), 1)
    corrs = torch.from_numpy(np.abs(coef*np.array(df.iloc[:, i]))-1).view(1, len(df)).type(torch.FloatTensor)
    loss += torch.mm(corrs, dist)/2
  loss_track.append(loss.item())
  if (epoch+1) % 5 == 0:
    print("{0}th epoch in process".format(epoch+1))
    print('running loss: {}'.format(loss.item()))
    print()

  optimizer.zero_grad()
  loss.backward(retain_graph=True)
  optimizer.step()

  with torch.no_grad():
    for element in embeddings:
      element.clamp_(0,1)

embeddings[:, 1]  

5th epoch in process
running loss: -623784.9375

10th epoch in process
running loss: -780746.625

15th epoch in process
running loss: -708773.6875

20th epoch in process
running loss: -734358.3125

25th epoch in process
running loss: -737078.5625

30th epoch in process
running loss: -760899.0625

35th epoch in process
running loss: -824675.4375

40th epoch in process
running loss: -823476.0625

45th epoch in process
running loss: -860724.75

50th epoch in process
running loss: -827216.625



tensor([0.8718, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0937, 0.0000,
        1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3677, 0.0000, 0.5608,
        0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.1229, 1.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0882, 0.0000], grad_fn=<SelectBackward>)

In [4]:
embeddings

tensor([[0.8020, 0.8718, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.7875, 1.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        [0.3970, 0.0882, 1.0000,  ..., 0.0000, 1.0000, 0.0000],
        [0.0000, 0.0000, 1.0000,  ..., 1.0000, 1.0000, 0.0000]],
       requires_grad=True)

## Rescaling Network

In [113]:
data = pd.read_csv("/content/drive/My Drive/stocks.csv")
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,stock_id,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume
0,000020,동화약품,2016-01-04,8130,8150,7920,8140,281440
1,000020,동화약품,2016-01-05,8040,8250,8000,8190,243179
2,000020,동화약품,2016-01-06,8200,8590,8110,8550,609906
3,000020,동화약품,2016-01-07,8470,8690,8190,8380,704752
4,000020,동화약품,2016-01-08,8210,8900,8130,8770,802330


In [114]:
data['KLength'] = data['closing_price'] - data['opening_price']
data['KUpperLength'] = data['highest_price'] - data[['opening_price', 'closing_price']].max(axis=1)
data['temp'] = data['opening_price']-data['lowest_price']
data['KLowerLength'] = data[['closing_price', 'temp']].min(axis=1)
data['return'] = data['closing_price'].diff()
data = data.drop(columns=['temp'])
data['stock_id'] = data['stock_id'].astype(str)
## 첫날 빼줘야됨 (종목 바뀔때 섞여들어갔다)
data.head()

,stock_id,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume,KLength,KUpperLength,KLowerLength,return
0,000020,동화약품,2016-01-04,8130,8150,7920,8140,281440,10,10,210,NaN
1,000020,동화약품,2016-01-05,8040,8250,8000,8190,243179,150,60,40,50.0
2,000020,동화약품,2016-01-06,8200,8590,8110,8550,609906,350,40,90,360.0
3,000020,동화약품,2016-01-07,8470,8690,8190,8380,704752,-90,220,280,-170.0
4,000020,동화약품,2016-01-08,8210,8900,8130,8770,802330,560,130,80,390.0


In [115]:
def year_parser(column):
  return int(column[0:4])

def month_parser(column):
  return int(column[5:7])

data['year'] = data['date'].apply(year_parser)
data['month'] = data['date'].apply(month_parser)

data = data[(data['year']==2016)&(data['month']<=6)]
data = data[data['stock_id'].isin(df.columns)]
# data = data.fillna(0)

KL = data.pivot(index='date', columns='stock_id', values='KLength')
R = data.pivot(index='date', columns='stock_id', values='return')

KL.shape

(121, 2030)

In [116]:
print(KL.shape, R.shape)

(121, 2030) (121, 2030)


In [117]:
KL = KL.iloc[:-1, ]
R = R.iloc[1:, ]

In [118]:
KL = KL.fillna(0)
R = R.fillna(0)

In [130]:
dtype = torch.FloatTensor
weights = Variable(torch.rand(1, 32).type(dtype), requires_grad=True)
lr = 0.05
optimizer = optim.SGD([weights], lr=lr)
epochs = 500
indic = np.array(KL)
returns = np.array(R)

for epoch in range(epochs):
  loss = 0

  rescale_fc = torch.exp(torch.mm(weights, embeddings)) / torch.sum(torch.exp(torch.mm(weights, embeddings)))
  opt_ind = torch.from_numpy(indic) * rescale_fc # element-wise
  # print('opt_ind: ', opt_ind)
  mean_ind = torch.mean(opt_ind, axis=0)
  # print('mean_ind: ', mean_ind)
  mean_ret = torch.mean(torch.from_numpy(returns), axis=0)
  # print('mean_ret: ', mean_ret)

  vx = opt_ind - mean_ind
  vy = torch.from_numpy(returns) - mean_ret
  # print('vx: ', vx)
  # print('vy: ', vy)
  loss = -torch.abs(torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2))))
  # print('loss: ', loss)

  if (epoch+1) % 50 == 0:
    print("{0}th epoch in process".format(epoch+1))
    print('running loss: {}'.format(loss.item()))
    print()

  optimizer.zero_grad()
  loss.backward(retain_graph=True)
  optimizer.step()

  with torch.no_grad():
    for weight in weights:
      weight.clamp_(0,1)

weights

50th epoch in process
running loss: -0.23575469466727395

100th epoch in process
running loss: -0.24138198184648657

150th epoch in process
running loss: -0.24291716706658018

200th epoch in process
running loss: -0.24359818579362438

250th epoch in process
running loss: -0.24400947203040615

300th epoch in process
running loss: -0.24428938605626754

350th epoch in process
running loss: -0.24449570638946883

400th epoch in process
running loss: -0.2446567266602514

450th epoch in process
running loss: -0.2447877880467779

500th epoch in process
running loss: -0.24489789899355893



tensor([[1.0787e-01, 4.5577e-01, 3.7170e-01, 1.8707e-04, 4.4074e-01, 8.1486e-01,
         2.2906e-01, 2.2651e-01, 1.6532e-01, 6.6833e-01, 3.3126e-01, 0.0000e+00,
         2.1989e-01, 4.7774e-01, 2.5917e-01, 4.1867e-01, 9.2924e-01, 5.3883e-01,
         6.7003e-01, 1.0000e+00, 1.7267e-01, 2.5216e-01, 7.6120e-01, 6.9733e-01,
         9.8284e-01, 1.6162e-01, 9.6101e-01, 9.1488e-01, 8.2262e-01, 6.1397e-01,
         1.5329e-01, 2.7696e-01]], requires_grad=True)